## [1] モデルをw2 * t_u ** 2 + w1 * t_u + bとして再定義してください

### （a）再定義を行うために、トレーニングループのどの部分を変更する必要がありますか？

In [72]:
import torch


t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)
t_un = t_u * 0.1

params = torch.tensor([1.0, 0.0])

In [91]:
def model(t_un, w, b):
    return w * t_un + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c) / t_p.size(0)

    return dsq_diffs

def get_grad(t_un, t_p, w):
    dloss_dtp = dloss_fn(t_p, t_c)
    dloss_dw = dloss_dtp * t_un
    dloss_db = dloss_dtp * 0.1
    
    return torch.stack([dloss_dw.sum(), dloss_db.sum()])

In [84]:
def training_loop(n_epochs, t_un, t_c, params, learning_rate,
                  print_params=True):
    
    for epoch in range(1, n_epochs + 1):
        
        w, b = params
        t_p = model(t_un, w ,b)
        loss = loss_fn(t_p, t_c)
        
        grad = get_grad(t_un, t_p, w)

        params = params - grad * learning_rate
        
        if epoch in [1, 2, 3, 10, 100, 1000, 2000, 3000, 5000]:
            print(f'Epoch {epoch}, Loss {float(loss)}')
            if print_params:
                print('    Params:', params)
                print('    Grad:  ', grad)
        
        if not torch.isfinite(loss).all():
            break  # <3>
    

In [92]:
training_loop(n_epochs = 1,
              t_un = t_un,
              t_c = t_c,
              params = params,
              learning_rate = 1e-2,
              print_params=True)

tensor([  6.1400, -16.8200, -18.3600, -39.6200, -10.7400,  -6.2200,   0.7800,
         12.3600,  -2.3200, -13.9200, -28.3200])
11
tensor([ 0.5582, -1.5291, -1.6691, -3.6018, -0.9764, -0.5655,  0.0709,  1.1236,
        -0.2109, -1.2655, -2.5745])
tensor(-10.6400)
tensor([-77.6140,  -1.0640])
Epoch 1, Loss 80.36434173583984
    Params: tensor([1.7761, 0.0106])
    Grad:   tensor([-77.6140,  -1.0640])
